<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">pyIMAGE</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);"><b style=color:red;>Image</b> analysis</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
<td><img style="height: 150px;" src="images/pyIMAGE.png"></td>
</tr>
</table>

----
# Image processing: PIL library ans Exif tags

In [4]:
import PIL
import PIL.Image
import PIL.ExifTags

In this notebook, we use the `PIL` library the read an image, and then recover the exif metadata in the
image and manipulate it.

We first define an image, then check with the external `file` command its content:
- PNG ([portable network graphics](https://de.wikipedia.org/wiki/Portable_Network_Graphics)) data format
- 4928 x 3264 pixel
- 8-bit depth
- 4 channels, red/green/(blue and an alpha channel (RGBA) 

In [20]:
#image file name
infile='images/Amrum.png'

!file images/Amrum.png

images/Amrum.png: PNG image data, 4928 x 3264, 8-bit/color RGBA, non-interlaced


----
## PIL library
We use the `PIL` (python image library) library to load, manipulate, andsave an image.

We first load the png image defined above into the `image` variable...

In [21]:
image = PIL.Image.open(infile)
print(image)

<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=4928x3264 at 0x10599EF90>


### Image properties
We check some image properties, e.g.
- width and height
- file name
- file description

In [22]:
print('size:               ',image.size)
width, height = image.size 
print('width,height:       ',width,height)
print('filename:           ',image.filename)
print('format:             ',image.format)
print('format_description: ',image.format_description)

size:                (4928, 3264)
width,height:        4928 3264
filename:            images/Amrum.png
format:              PNG
format_description:  Portable network graphics


### Resizing
We resize the image, then rotate it

In [23]:
newsize = (int(width/4),int(height/4))
image = image.resize(newsize)
print(image)

<PIL.Image.Image image mode=RGBA size=1232x816 at 0x106025F90>


### Displaying
We display the image, both with an external viewer, and in the notebook:

In [ ]:
# show image with external viewerx
image.show()
# show image inline
display(image)

### Converting
We change the `png` image to `jpg`. As jpg files have no transparency channel, we first need to convert the image
from `RGBA` to `RGB`, then save it as jpg.

In [24]:
if image.mode != 'RGB':
    image = image.convert('RGB')
image.save('images/Amrum.jpg','jpeg')
print(image)

<PIL.Image.Image image mode=RGB size=1232x816 at 0x1059A9B90>


### Extracting `exif` metadata 
We use the `ExifTags` package to extract exif information from the image.

The exif metadata are stored in a special block at the beginning of the image file. 

The `tagid` is basically the pointer to the starting block of a specific information in the metadata block.

In [25]:
# extracting the exif metadata
exifdata = image.getexif()
 
# looping through all the tags present in exifdata
for tagid in exifdata:
     
    # getting the tag name instead of tag id
    tagname = PIL.ExifTags.TAGS.get(tagid, tagid)
 
    # passing the tagid to get its respective value
    tagvalue = exifdata.get(tagid)
   
    # printing the final result
    print(f"{tagid:10}: {tagname:25}: {tagvalue}")

     34853: GPSInfo                  : 35496
       296: ResolutionUnit           : 2
     34665: ExifOffset               : 228
       271: Make                     : NIKON CORPORATION
       272: Model                    : NIKON D5100
       305: Software                 : Ver.1.01 
       274: Orientation              : 1
       306: DateTime                 : 2018:10:03 15:19:14
       531: YCbCrPositioning         : 2
       282: XResolution              : 300.0
       283: YResolution              : 300.0


### Changing exif metadata
We now change some exif data.

We first read selected exif metadata by using a list of `tagid` values. Note
that `Artist` and `Copyright` are empty ..

In [26]:
for tagid in [271,272,306,36867,315,33432,37510]:
    tagname = PIL.ExifTags.TAGS.get(tagid, tagid)
    tagvalue = exifdata.get(tagid)
    print(tagid,tagname,tagvalue)

271 Make NIKON CORPORATION
272 Model NIKON D5100
306 DateTime 2018:10:03 15:19:14
36867 DateTimeOriginal None
315 Artist None
33432 Copyright None
37510 UserComment None


We add `Artist` and `Copyright` information by changing the selected exif tags in the exif data set read earlier, 
then save the updated `exifdata` object to a new file.

In [27]:
exifdata[315] = 'Georg Kaufmann'
exifdata[33432] = '(c) Georg Kaufmann'
exifdata[37510] = 'xmin,ymin,xmax,ymax: 11,12,13,14 dx,dy: xx'

output_file = 'images/out.png'
image.save(output_file, exif=exifdata)

We check the new image exif data ...

In [33]:
image2 = PIL.Image.open('images/out.png')
# extracting the exif metadata
exifdata = image2.getexif()
 
# looping through all the tags present in exifdata
for tagid in exifdata:
     
    # getting the tag name instead of tag id
    tagname = PIL.ExifTags.TAGS.get(tagid, tagid)
 
    # passing the tagid to get its respective value
    tagvalue = exifdata.get(tagid)
   
    # printing the final result
    print(f"{tagid:10}: {tagname:25}: {tagvalue}")

     34853: GPSInfo                  : 35560
     37510: UserComment              : xmin,ymin,xmax,ymax: 11,12,13,14 dx,dy: xx
       296: ResolutionUnit           : 2
     34665: ExifOffset               : 294
       271: Make                     : NIKON CORPORATION
       272: Model                    : NIKON D5100
       305: Software                 : Ver.1.01 
       274: Orientation              : 1
       306: DateTime                 : 2018:10:03 15:19:14
       531: YCbCrPositioning         : 2
       283: YResolution              : 300.0
     33432: Copyright                : (c) Georg Kaufmann
       282: XResolution              : 300.0
       315: Artist                   : Georg Kaufmann


In [32]:
print(PIL.ExifTags.Base.UserComment.value)
print(PIL.ExifTags.Base.UserComment.name)

37510
UserComment


In [ ]:
PageNumber = 0x0129
print(type(PageNumber),PageNumber)

In [ ]:
from PIL.ExifTags import GPS

GPS.GPSDestLatitude.name

----
## Exif format
See [manual](https://www.media.mit.edu/pia/Research/deepview/exif.html)

Every JPEG file starts from binary value '0xFFD8', ends by binary value '0xFFD9'. There are several binary 0xFFXX data in JPEG data, they are called as "Marker", and it means the period of JPEG information data. 0xFFD8 means SOI(Start of image), 0xFFD9 means EOI(End of image). These two special Markers have no data following, the other Markers have data with it. 

In [ ]:
# start of image data
decimal=65496
hexadecimal = hex(65496)
binary = bin(65496)
print('decimal:     ',decimal)
print('hexadecimal: ',hexadecimal)
print('binary:      ',binary)

# end of image data
hexadecimal = '0xFFD9'
print('hexadecimal: ',hexadecimal)
print('decimal:      ',int(hexadecimal,16))

# exif: UserComment
hexadecimal = '0x9286'
print('hexadecimal: ',hexadecimal)
print('decimal:      ',int(hexadecimal,16))

[stackoverflow1](https://stackoverflow.com/questions/32085948/python-pil-exiftags-not-sure-what-it-is-all-about)

[lon/lat](https://gist.github.com/erans/983821/e30bd051e1b1ae3cb07650f24184aa15c0037ce8)

In [ ]:
def get_exif_data(image):
    """
    Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags
    """
    exif_data = {}
    info = image._getexif()
    
    if info:
        for tag, value in info.items():
            decoded = PIL.ExifTags.TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for t in value:
                    sub_decoded = PIL.ExifTags.GPSTAGS.get(t, t)
                    gps_data[sub_decoded] = value[t]

                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value

    return exif_data

exif_data = get_exif_data(image)
#print(exif_data)

info = image._getexif()
print(info[37510])
print(hex(37510))
print(info[0x9286])

[first working one](https://blog.matthewgove.com/2022/05/13/how-to-bulk-edit-your-photos-exif-data-with-10-lines-of-python)

In [ ]:
PILLOW_TAGS = [
    271,    # Camera Make
    272,    # Camera Model
    36867,  # Date/Time Photo Taken
    315,     # Artist Name
    33432,   # Copyright Message
    34853,  # GPS Info
]

EXIF_TAGS = [
    "make",
    "model",
    "DateTimeOriginal",
    "artist",
    "copyright",
    "gps_latitude",
    "gps_latitude_ref",
    "gps_longitude",
    "gps_longitude_ref",
    "gps_altitude",
]

pillow_img = PIL.Image.open(infile)
img_exif = pillow_img.getexif()

for tag in PILLOW_TAGS:
    print(tag)
    try:
        english_tag = PIL.ExifTags.TAGS[tag]
        value = img_exif[tag]
    except:
        continue
    print("{}: {}".format(english_tag, value))

In [ ]:
PILLOW_TAGS = [
    315,     # Artist Name
    33432,   # Copyright Message
]

EXIF_TAGS = [
    "artist",
    "copyright"
]

VALUES = [
    "Matthew Gove",    # Artist Name
    "Copyright 2022 Matthew Gove. All Rights Reserved."  # Copyright Message
]

In [ ]:
pillow_img = PIL.Image.open(infile)
img_exif = pillow_img.getexif()
for tag, value in zip(PILLOW_TAGS, VALUES):
    img_exif[tag] = value
    
output_file = 'out.png'
pillow_img.save(output_file, exif=img_exif)

In [ ]:
PILLOW_TAGS = [
    271,    # Camera Make
    272,    # Camera Model
    36867,  # Date/Time Photo Taken
    315,     # Artist Name
    33432,   # Copyright Message
    34853,  # GPS Info
]

EXIF_TAGS = [
    "make",
    "model",
    "DateTimeOriginal",
    "artist",
    "copyright",
    "gps_latitude",
    "gps_latitude_ref",
    "gps_longitude",
    "gps_longitude_ref",
    "gps_altitude",
]

pillow_img = PIL.Image.open('out.png')
img_exif = pillow_img.getexif()

for tag in PILLOW_TAGS:
    try:
        english_tag = PIL.ExifTags.TAGS[tag]
        value = img_exif[tag]
    except:
        continue
    print("{}: {}".format(english_tag, value))

In [ ]:
def _get_if_exist(data, key):
    if key in data:
        return data[key]
    return None

In [ ]:
def get_lat_lon(exif_data):
    """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
    lat = None
    lon = None

    if "GPSInfo" in exif_data:		
        gps_info = exif_data["GPSInfo"]

        gps_latitude = _get_if_exist(gps_info, "GPSLatitude")
        gps_latitude_ref = _get_if_exist(gps_info, 'GPSLatitudeRef')
        gps_longitude = _get_if_exist(gps_info, 'GPSLongitude')
        gps_longitude_ref = _get_if_exist(gps_info, 'GPSLongitudeRef')

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            lat = _convert_to_degress(gps_latitude)
            if gps_latitude_ref != "N":                     
                lat = 0 - lat

            lon = _convert_to_degress(gps_longitude)
            if gps_longitude_ref != "E":
                lon = 0 - lon

    return lat, lon

lat,lon = get_lat_lon(exif_data)
print(lat,lon)

In [18]:
image = PIL.Image.open(infile)

info = image._getexif()
keys = list(info.keys())
print('keys: ',keys)

print(keys[0],PIL.ExifTags.TAGS[keys[0]])

keys:  [34853, 296, 34665, 271, 272, 305, 274, 306, 531, 282, 283, 36864, 37121, 37122, 36867, 36868, 37380, 37381, 37383, 37384, 37385, 37386, 37510, 40961, 40962, 40965, 41990, 37520, 37521, 37522, 40963, 41996, 41495, 41728, 33434, 33437, 41729, 34850, 41730, 41985, 34855, 41986, 40960, 34864, 41987, 41988, 41989, 41991, 41992, 41993, 41994, 37500]
34853 GPSInfo


In [19]:
import io
image = PIL.Image.open(infile)

info = image._getexif()
if info:
    for tag, value in info.items():
        decoded = PIL.ExifTags.TAGS.get(tag, tag)
        for itag in ['Model','DateTimeOriginal','XResolution','YResolution','FocalLength','ExposureTime','MaxApertureValue','ExposureMode','ExifImageWidth',
                     'ExifImageHeight','UserComment','GPSInfo']:
            if (decoded==itag):
                print(decoded,':',tag,value)
        if (decoded == "GPSInfo"):
            gps_data = {}
            for t in value:
                gps_decoded = PIL.ExifTags.GPSTAGS.get(t, t)
                gps_data[gps_decoded] = value[t]
print(gps_data)

GPSInfo : 34853 {0: b'\x02\x03\x00\x00'}
Model : 272 NIKON D5100
XResolution : 282 300.0
YResolution : 283 300.0
DateTimeOriginal : 36867 2018:10:03 15:19:14
MaxApertureValue : 37381 5.0
FocalLength : 37386 105.0
UserComment : 37510 b'ASCII\x00\x00\x00                                    '
ExifImageWidth : 40962 4928
ExifImageHeight : 40963 3264
ExposureTime : 33434 0.0015625
ExposureMode : 41986 0
{'GPSVersionID': b'\x02\x03\x00\x00'}


In [ ]:
if info:
    for tag, value in info.items():
        decoded = PIL.ExifTags.TAGS.get(tag, tag)
        for itag in ['UserComment']:
            if (decoded==itag):
                print(decoded,':',tag,value)
print(info[0x9286])     

In [ ]:
#print(info)
info = image._getexif()
info[0x9286] = "test"
image.save("out.png", exif=info)

In [ ]:
image1 = PIL.Image.open(infile)
info = image1._getexif()
print(str(info[0x9286].decode()))

In [ ]:
image1 = PIL.Image.open(infile)
exif = image1.getexif()
exif[0x9286] = "test"
image1.save("out.png", exif=exif)

In [ ]:
image2 = PIL.Image.open('out.png')
info = image2._getexif()
print(str(info[0x9286].decode()))

In [ ]:
exif = image.getexif()
exif[0x9286] = "test"
image.save("out.png", exif=exif)

In [ ]:
image.info.keys()

In [ ]:
image.info['dpi'][:30]

In [ ]:
exifd = img._getexif()
keys = list(exifd.keys())
print(keys)

keys = [k for k in keys if k in TAGS]
print("\n".join([TAGS[k] for k in keys]))

gpsinfo = exifd[_TAGS_r["GPSInfo"]]
print(gpsinfo.keys())
print("\n".join([str((GPSTAGS[k], gpsinfo[k])) for k in gpsinfo.keys()]))

The 8-bit depth of the color space results in
$$
2^8=256
$$
different colors per channel, normally addressed with the range [0,255], thus 256 integer numbers.

A usual color image has, for each pixel, a triple (rgb) of a quadruple (rgba) of
integer points for the 
<font style="color:red;font-weight: bold;">red</font>, 
<font style="color:green;font-weight: bold;">green</font>,
<font style="color:blue;font-weight: bold;">blue</font>
color space, and, if present, and integer for the transparency 
<font style="color:black;font-weight: bold;">a</font>.

----
## Loading
Next, we load the png image with two libraries:
- `imread` from the `matplotlib.image`library, opens png format natively, all other formats using the `PIL` library
- `Image.open` from the `PIL` library.

###  `imread`

After reading the file with the `imread` function from matplotlib, we receive a
3D numpy array, with **floating-point** numbers, the first and second dimension the 
pixel rows and columns, and the third dimension the color (rgb) and tansparency
(a) channels.

**Note:** While all other image formats result in an int8 array, only the png file format
is natively read in as float, scaled between [0,1] instead of [0,255].

In [ ]:
# load image 
image = mpimg.imread(infile)
print (type(image),image.ndim,image.dtype,image.shape)
print(image[0][0])

# show image
fig,axs = plt.subplots(1,1)
axs.imshow(image)

###  `Image.read`

After reading the file with the `Image.read` function from PIL, we receive a
PIL object, whose features can be analysed. e.g. with `.format`, `.size`, or `.mode`.

For using our own image manipulation within numpy and scipy, we need to convert
the PIL object first to a numpy array, using `asarray`.


Then we obtain a 3D numpy array, with **integer** numbers, the first and second dimension the 
pixel rows and columns, and the third dimension the color (rgb) and tansparency
(a) channels.

**Note:** Now, the the png file format
is integer within the range [0,255].

In [ ]:
image = PIL.Image.open(infile)
print (image.format,image.size,image.mode)
# convert to ndarray
image_arr = np.asarray(image)
print (type(image_arr),image_arr.dtype,image_arr.ndim,image_arr.shape)
print(image_arr[0][0])

# show image
fig,axs = plt.subplots(1,1)
axs.imshow(image_arr)

----